<a href="https://colab.research.google.com/github/mdsaifhaider/Bag-of-Words-Meets-Bags-of-Popcorn/blob/main/Bag_of_Words_Meets_Bags_of_Popcorn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##kaggle score of 0.83472
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
id=[]
sentiment=[]
review=[]

In [ ]:
import pandas as pd
data=pd.read_csv("./labeledTrainData.tsv",delimiter="\t",header=0,quoting=3)
data.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [ ]:
data["id"]=data["id"].str.replace('"','').str.replace("'","")
sentiment=data["sentiment"]
review=data["review"]

In [ ]:
data["review"]=data["review"].str.replace("<br>","").str.replace("<br />","").str.replace('"','')

In [ ]:
review=data["review"]

In [ ]:
review[1]

"\\The Classic War of the Worlds\\ by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for different things in a movie. Those who envision themselves as amateur \\critics\\ look only to criticize everything they can. Others rate a movie on more important bases,like being entertained, which is why most people never agree with the \\critics\\. We enjoyed the effort Mr. Hines put into being faithful to H.G. Wells' classic novel, and we found it to be very entertaining. This made it easy to overlook what the \\critics\\ perceive to be its shortcomings."

In [ ]:
data.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,\The Classic War of the Worlds\ by Timothy Hin...
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [ ]:
split=int(0.9*len(review))
split

22500

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer=Tokenizer(oov_token="<OOV>")

In [ ]:
tokenizer.fit_on_texts(review)
word_index=tokenizer.word_index
total_word=len(word_index)

In [ ]:
sequences=tokenizer.texts_to_sequences(review)

In [ ]:
max_len=max([len(x) for x in sequences])

In [ ]:
max_len

2466

In [ ]:
train_data=review[:split]
train_padded=pad_sequences(tokenizer.texts_to_sequences(train_data),padding="post",maxlen=max_len)

In [ ]:
validation_data=review[split:]
validation_padded=pad_sequences(tokenizer.texts_to_sequences(validation_data),padding="post",maxlen=max_len)

In [ ]:

train_label=sentiment[:split]
validation_label=sentiment[split:]


In [ ]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Embedding(total_word+1,100,input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(word_index)/2,activation="relu",kernel_regularizer=tf.keras.regularizers.l2(0.003)),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

In [ ]:
model.compile(optimizer='adam',loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
train_label=np.array(train_label)
train_padded=np.array(train_padded)
validation_label=np.array(validation_label)
validation_padded=np.array(validation_padded)

In [ ]:
model.fit(train_padded,
          train_label,
          validation_data=(validation_padded,validation_label),
          epochs=10)

Epoch 1/10
704/704 [==============================] - 3032s 4s/step - loss: 0.5267 - accuracy: 0.7547 - val_loss: 0.4672 - val_accuracy: 0.8312
Epoch 2/10
704/704 [==============================] - 2856s 4s/step - loss: 0.5584 - accuracy: 0.7277 - val_loss: 0.3956 - val_accuracy: 0.8500
Epoch 3/10
704/704 [==============================] - 2993s 4s/step - loss: 0.3264 - accuracy: 0.8661 - val_loss: 0.3833 - val_accuracy: 0.8640
Epoch 4/10
704/704 [==============================] - 2887s 4s/step - loss: 0.1781 - accuracy: 0.9421 - val_loss: 0.4065 - val_accuracy: 0.8316
Epoch 5/10
704/704 [==============================] - 2851s 4s/step - loss: 0.2302 - accuracy: 0.9128 - val_loss: 0.5427 - val_accuracy: 0.7900
Epoch 6/10
704/704 [==============================] - 2876s 4s/step - loss: 0.1401 - accuracy: 0.9613 - val_loss: 0.4198 - val_accuracy: 0.8688
Epoch 7/10
704/704 [==============================] - 2902s 4s/step - loss: 0.0732 - accuracy: 0.9805 - val_loss: 0.4412 - val_accuracy:

In [ ]:
data=pd.read_csv("./testData.tsv",delimiter="\t",quoting=3,header=0)

In [ ]:
review=data["review"]
review[0]

'"Naturally in a film who\'s main themes are of mortality, nostalgia, and loss of innocence it is perhaps not surprising that it is rated more highly by older viewers than younger ones. However there is a craftsmanship and completeness to the film which anyone can enjoy. The pace is steady and constant, the characters full and engaging, the relationships and interactions natural showing that you do not need floods of tears to show emotion, screams to show fear, shouting to show dispute or violence to show anger. Naturally Joyce\'s short story lends the film a ready made structure as perfect as a polished diamond, but the small changes Huston makes such as the inclusion of the poem fit in neatly. It is truly a masterpiece of tact, subtlety and overwhelming beauty."'

In [ ]:
data["review"]=data["review"].str.replace("<br>","").str.replace("<br />","").str.replace('"','')

In [ ]:
review[7]

"This is one of those movies I watched, and wondered, why did I watch it? What did I find so interesting about it? Being a truck driver myself, I didn't find it very realistic. No, I've never used a 'lot lizard', nor have I ever seen, nor heard about one traveling around the country in a brand new seventy thousand dollar RV, either.Same thing about a pimp whom has never sampled the lady in question (until the end of the movie, and well, he still really didn't...), and only getting 50 bucks 'a cut', when the prostitute gets $200.00 (well, $150.00 after his cut, yeah...).I still laugh at the lot lizard comment Ivey made (them's Lot Lizards, they'll screw anything with 20 bucks, and some are men dressed as woman... or something equally as weird), meaning, we're better then them, as we may still be prostitutes, but we get paid BETTER.Other then that, it's just a story of a young woman whom wanted something more from life then a dead end job while living at home (she's 18, remember?) and em

In [ ]:
test_sequence=tokenizer.texts_to_sequences(review)

In [ ]:
pad_test=pad_sequences(test_sequence,padding="post",maxlen=max_len)

In [ ]:
pad_test=np.array(pad_test)

In [ ]:
prediction=model.predict(pad_test)

In [ ]:
round_prediction=np.round(prediction)

In [ ]:
round_prediction[16000]

array([1.], dtype=float32)

In [ ]:
data["id"]=data["id"].str.replace('"','').str.replace("'","")
id=data[["id"]].values
id

array([['12311_10'],
       ['8348_2'],
       ['5828_4'],
       ...,
       ['2531_1'],
       ['7772_8'],
       ['11465_10']], dtype=object)

In [ ]:
final_id=id.tolist()

In [ ]:
final_sentiment=round_prediction.tolist()
final_sentiment

[[1.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [1.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [1.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [1.0],
 [0.0],
 [1.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [1.0],
 [1.0],
 [1.0],
 [0.0],
 [1.0],
 [1.0],


In [ ]:
final_id

[['12311_10'],
 ['8348_2'],
 ['5828_4'],
 ['7186_2'],
 ['12128_7'],
 ['2913_8'],
 ['4396_1'],
 ['395_2'],
 ['10616_1'],
 ['9074_9'],
 ['9252_3'],
 ['9896_9'],
 ['574_4'],
 ['11182_8'],
 ['11656_4'],
 ['2322_4'],
 ['8703_1'],
 ['7483_1'],
 ['6007_10'],
 ['12424_4'],
 ['4672_1'],
 ['10841_3'],
 ['8954_7'],
 ['7392_1'],
 ['10288_8'],
 ['5343_4'],
 ['4950_1'],
 ['9257_4'],
 ['8689_3'],
 ['4480_2'],
 ['9286_10'],
 ['12245_10'],
 ['7529_7'],
 ['4_4'],
 ['5299_2'],
 ['12077_8'],
 ['10701_2'],
 ['11579_1'],
 ['10993_1'],
 ['8913_1'],
 ['1350_3'],
 ['3804_2'],
 ['5025_3'],
 ['5170_8'],
 ['8225_8'],
 ['6427_7'],
 ['3165_3'],
 ['7902_9'],
 ['8208_3'],
 ['3577_8'],
 ['4865_1'],
 ['9063_3'],
 ['1106_1'],
 ['1117_9'],
 ['1050_4'],
 ['10755_9'],
 ['2729_8'],
 ['10072_10'],
 ['2904_7'],
 ['4584_2'],
 ['189_4'],
 ['6302_4'],
 ['8138_9'],
 ['1262_1'],
 ['2199_10'],
 ['6551_7'],
 ['8082_7'],
 ['4897_8'],
 ['7360_1'],
 ['11012_10'],
 ['12046_1'],
 ['2331_2'],
 ['8961_9'],
 ['751_7'],
 ['9884_10'],
 ['1001

In [ ]:
final_dictionary={"id":final_id,"sentiment":final_sentiment}
df=pd.DataFrame(final_dictionary)

In [ ]:
df["id"]=df["id"].str[0]
df["sentiment"]=df["sentiment"].str[0]

In [ ]:
df

,id,sentiment
0,12311_10,1.0
1,8348_2,0.0
2,5828_4,1.0
3,7186_2,1.0
4,12128_7,1.0
...,...,...
24995,2155_10,1.0
24996,59_10,1.0
24997,2531_1,0.0
24998,7772_8,1.0


In [ ]:
df.to_csv("file.csv",index=False)